In [2]:
import pandas as pd
from prophet import Prophet
import numpy as np
import holidays
from prophet.diagnostics import cross_validation
import pandas as pd
import numpy as np
import itertools
from datetime import date
import holidays
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric

Importing plotly failed. Interactive plots will not work.


In [7]:
def get_df(partner):
    df = pd.read_csv('partners-table.csv')
    df = df.drop('month', axis=1)
    df = df.drop('Unnamed: 4', axis=1)
    df = df[(df['merchant_name'] == partner)]
    df = df.drop('merchant_name', axis=1)
    df = df.reset_index(drop=True)
    df = df.rename(columns={"day": "ds", "cashback":"y"})
    df['ds'] = pd.to_datetime(df['ds'])
    df['day_of_week'] = df['ds'].dt.dayofweek
    return df

In [8]:
# Вкидываем праздники, для их учёта моделькой
holidays_dict = holidays.RU(years=(2022, 2023))
df_holidays = pd.DataFrame.from_dict(holidays_dict, orient='index') \
    .reset_index()
df_holidays = df_holidays.rename({'index':'ds', 0:'holiday'}, axis ='columns')
df_holidays['ds'] = pd.to_datetime(df_holidays.ds)
df_holidays = df_holidays.sort_values(by=['ds'])
df_holidays = df_holidays.reset_index(drop=True)
print(df_holidays)

           ds                      holiday
0  2022-01-01            New Year Holidays
1  2022-01-02            New Year Holidays
2  2022-01-03            New Year Holidays
3  2022-01-04            New Year Holidays
4  2022-01-05            New Year Holidays
5  2022-01-06            New Year Holidays
6  2022-01-07                Christmas Day
7  2022-01-08            New Year Holidays
8  2022-02-23    Fatherland Defender's Day
9  2022-03-08    International Women's Day
10 2022-05-01  Holiday of Spring and Labor
11 2022-05-09                  Victory Day
12 2022-06-12                   Russia Day
13 2022-11-04                    Unity Day
14 2023-01-01            New Year Holidays
15 2023-01-02            New Year Holidays
16 2023-01-03            New Year Holidays
17 2023-01-04            New Year Holidays
18 2023-01-05            New Year Holidays
19 2023-01-06            New Year Holidays
20 2023-01-07                Christmas Day
21 2023-01-08            New Year Holidays
22 2023-02-

In [9]:
# Кол-во дней, которые надо отрезать и предсказать
predictions = 30

In [10]:
# Отрезаем из обучающей выборки последние N точек, чтобы измерить на них качество
train_df = df

NameError: name 'df' is not defined

In [11]:
def get_best_params(df):

    # Покрутим разные комбинации гиперпараметров
    dif_days = int((pd.to_datetime(df["ds"].max()) - pd.to_datetime(df["ds"].min())).days) // 2
    param_grid = {
        'changepoint_prior_scale': [0.25, 0.05, 0.1], ## по умолчанию 0.05, попробуем увеличить и уменьшить в два раза
        'seasonality_prior_scale': [5.0, 10.0, 20.0], ## по умолчанию 10.0, попробуем увеличить и уменьшить в два раза
        'holidays_prior_scale': [5.0, 10.0, 20.0], ## по умолчанию 10.0, попробуем увеличить и уменьшить в два раза
    }
    
    # Создаем все комбинации параметров
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    mapes = []  # Сюда будем складывать метрику MAPE
    
    # Крутим кроссвалидацию со всеми комбинациями параметров
    for params in all_params:
        m = Prophet(**params, holidays=df_holidays, daily_seasonality="auto", weekly_seasonality="auto", yearly_seasonality="auto")
        m.add_regressor('day_of_week')
        m.fit(df)  # Fit model with given params
        df_cv = cross_validation(m, initial=f'{dif_days} days', period=f'{dif_days//6} days', horizon = f'{dif_days//3 } days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1) ## тут окно для подсчета метрики 1, чтобы метрика считалась по всему горизонту
        mapes.append(df_p['mape'].values[0])
    
    # Смотрим на результаты с разными параметрами
    tuning_results = pd.DataFrame(all_params)
    tuning_results['mape'] = mapes
    return all_params[np.argmin(mapes)]
    

In [12]:
# # Отображаем лучшие параметры
# best_params = all_params[np.argmin(mapes)]
# print(best_params)
# print(best_params["changepoint_prior_scale"])

In [27]:
def predict(df, days):
    m = Prophet(holidays=df_holidays, daily_seasonality="auto", weekly_seasonality="auto", yearly_seasonality="auto")
                # changepoint_prior_scale=best_params["changepoint_prior_scale"], seasonality_prior_scale=best_params["seasonality_prior_scale"], holidays_prior_scale=best_params["holidays_prior_scale"])
    m.add_regressor('day_of_week')
    m.fit(df)
    future = df[['ds']].copy()
    future = m.make_future_dataframe(days)
    future['day_of_week'] = future['ds'].dt.dayofweek
    forecast = m.predict(future)
    return forecast
    # forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    

In [49]:
def get_end_date(partner, ndf, df):
    with open('partners.json') as f:
        partners = json.load(f)
    l = []
    c = 10
    while int(ndf["trend"].sum()) < int(partner["budget"]):
        c += 20
        ndf = predict(df, c)
        l = ndf[["ds", "trend"]]
    s = 0
    for i in l["trend"]:
        if s <= int(partner["budget"]):
            s += i
        else:
            partner["stoped_date"] = str(l.loc[l['trend'] == i, 'ds'])
            break
    with open('partners.json', 'w') as file:
        json.dump(partners, file)
    
    

In [ ]:
import json
from flask import Flask, request, jsonify, render_template

with open('partners.json') as f:
    partners = json.load(f)
    
    

    



def get_partner_by_id(id): #получение партнера по id
    for partner in partners["partners"]:
        if partner["id"] == id:
            return partner
    return jsonify({"error": "Партнер с данным ID не найден"}), 404


app = Flask(__name__)


@app.route('/')
def index():
    return render_template("index.html")

@app.route('/api/partners', methods=['POST']) #Создание партнеров с бюджетами
def create_partner():
    partner_name = request.args.get('name')
    partner_budget = request.args.get('budget')
    df = get_df(partner_name)
    #best_params = get_best_params(df)
    ndf = predict(df, 10)
    get_end_date(partner_name, ndf, df)
    
    partner = {
        "id": len(partners["partners"]) + 1,
        "name": partner_name,
        "budget": partner_budget,
        "spent_budget": 0,
        "best_params": best_params,
        "is_stopped": "False",
    }

    partners["partners"].append(partner)
    with open('partners.json', 'w') as file:
        json.dump(partners, file)
    return jsonify(partner), 200


@app.route('/api/partners/<int:id>', methods=['GET']) # Получение информации о партнере
def get_partner(id):
    get_partner_by_id(id)


@app.route('/api/partners/<int:id>/cashback', methods=['PUT']) #обновление данных о выплаченном кэшбэке
def update_cashback(id):
    data = request.json
    date = data.get("date")
    date = pd.to_datetime(date)
    cashback_amount = data.get("cashback")
    partner = get_partner_by_id(id)
    print(partner["stoped_date"])
    date_str = partner["stoped_date"].split("\n")[0].split("   ")[1].split("-")
    #stopped_date = date(int(date_str[0]), int(date_str[1]), int(date_str[2]))
    s7
    #if date >= stopped_date: # partner["stoped_date"]:
        #partner["is_stopped"] = "True"
    
    partner['spent_budget'] = cashback_amount
    df = get_df(partner["name"])
    df = df._append({"ds": date, "y": cashback_amount, "day_of_week": 0}, ignore_index=True)     
    ndf = predict(df, 10)
    get_end_date(partner, ndf, df)
    
    
    
    
    with open('partners.json', 'w') as file:
        json.dump(partners, file)
    return jsonify(partner), 200


    


if __name__ == '__main__':
  app.run(port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
[2024-02-25 14:49:52,208] ERROR in app: Exception on /api/partners/3/cashback [PUT]
Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 870, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 855, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type:

316   2023-11-17
Name: ds, dtype: datetime64[ns]
